In [ ]:
from dataset import tweetProccess
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


'''gets the data'''
dataHandler = tweetProccess.FileTreat()
content = dataHandler.getBadTweets()

no_features = 2000


#Use to download nltk content
#nltk.download()

'''defining a function to remove numbers'''
def tokenize(text):
    tokenized = []
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', word):
            tokenized.append(word.lower())
            
    return tokenized

'''defining a function to tokenize and get only vebs and adjectivies'''
def tokenizeStructures(text):
    #gets the type of each word in the text
    tags = pos_tag(word_tokenize(text))
    
    interesting = []
    for tag in tags:
        #removes tweet links
        if re.search('t.co+', tag[0]):
            continue
        
        if (tag[1] == 'VERB' or tag[1] == 'JJ') and len(tag[0]) > 2:
            interesting.append(tag[0])
    
    return interesting

def getLda(dataFrame, no_clusters):
    ldas = []
    no_topics = 7
    
    #for each cluster
    print(no_clusters)
    for cluster in range(no_clusters):
        print('cluster: ' + str(cluster))
        
        lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
        tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')
        
        #get text of tweets from this cluster
        texts = dataFrame['text'][cluster].values
        
        #creates a tf matrix for this cluster
        tf = tf_vectorizer.fit_transform(texts)
        
        #get feature names for this matrix
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        #extract latent topics
        ldas.append((cluster, lda.fit(tf), tf_feature_names))
        
    return ldas
        
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

def print_top_words(model, feature_names, n_top_words):
    #print('model size:' + str(len(model.components_)))
    for topic_idx, topic in enumerate(model.components_):
        #print(feature_names)
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[ -n_top_words:]])
        print(message)

In [ ]:
from sklearn.cluster import KMeans

#generate the tf matrix
tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(content)

#generates the clusters
n_cluters = 4
km = KMeans(n_clusters = n_cluters)
%time km.fit(tf)

clusters = km.labels_.tolist()

In [ ]:
import pandas as pd

table = {'text': content, 'cluster': clusters, 'id': [idx for idx in range(len(content))]}

dataframe = pd.DataFrame(table, index=[clusters], columns=['text', 'cluster', 'id'])

#sizes of each cluster
print(dataframe['cluster'].value_counts())

#gets the latent topic of each cluster
result = getLda(dataframe, n_cluters)

In [ ]:
no_top_words = 10

for cluster, lda, feature_names in result:
    #print(len(lda.components_[0]))
    if(len(feature_names) > 10):
        
        print("cluster: " + str(cluster))
        print_top_words(lda, feature_names, 10)
        print('================================================')

In [ ]:
print(enumerate(result[3][1].components_))

In [ ]:
dataframe.tail()

In [ ]:
dataframe['cluster'].value_counts()
dataframe.tail()


In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
#dist = 1 - cosine_similarity(tf)

#print(dist)